# Evaluation of SIRET index on French RESTAURANTS open database

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from worksites import WorkSiteName, GeoWorkSiteName, geocode_worksites
from queries import request_elastic
import pandas as pd
import os
import elasticsearch as es

In [ ]:
try:
    os.remove("export_alimconfiance.csv")
except:
    pass
!wget -c "https://www.data.gouv.fr/fr/datasets/r/fff0cc27-977b-40d5-9c11-f7e4e79a0b72" -O export_alimconfiance.csv

In [ ]:
# Preparing evaluatest dataset (database of French restaurants)

In [ ]:
size_test = 100
df = pd.read_csv("export_alimconfiance.csv", sep=";", dtype=str)[["APP_Libelle_etablissement", "SIRET", "Libelle_commune", "Code_postal", "Adresse_2_UA", "geores"]]

In [ ]:
df = df[(df["Code_postal"].str[:2] != "95") & (df["Code_postal"].str[:2] != "94")]

In [ ]:
df = df[10:11]

In [ ]:
wks = [
    WorkSiteName(**{
        "siret":row["SIRET"],
        "name": row["APP_Libelle_etablissement"],
        "postcode": row["Code_postal"],
        "cityname": row["Libelle_commune"], 
        "address": row["Adresse_2_UA"]
    })
    for k, row in df.head(size_test).iterrows()
]

In [ ]:
geowks = geocode_worksites(wks)

In [ ]:
ELASTIC_URL = "http://elasticsearch-master:9200"
conn = es.Elasticsearch(ELASTIC_URL)

In [ ]:
# Check if the first response of Elasticsearch

In [ ]:
geo_threshold = 0.7

In [ ]:
def check_if_same_score(output):
    if output:
        filter_output = []
        score = output[0]["_score"]
        for out in output:
            if out["_score"] == score:
                filter_output.append(out)
            else:
                return filter_output
    else:
        return []

In [ ]:
geowk.name = "LIDL"

In [ ]:
geowk

In [ ]:
def get_answer(conn, geoworksite, siret_truth:int, geo_threshold: float):
    output = request_elastic(conn, geowk, geo_threshold=geo_threshold, use_geo=True)
    output = check_if_same_score(output)
    if output:
        for out in output:
            if out["_source"]["siret"] == siret_truth:
                return out
    else:
        output = request_elastic(conn, geowk, use_geo=False)
        output = check_if_same_score(output)
        if output:
            for out in output:
                if out["_source"]["siret"] == siret_truth:
                    return out
        else:
            return

def all_relevant_elastic_answers(conn, geoworksite, geo_threshold:float):
    output = request_elastic(conn, geowk, geo_threshold=geo_threshold)


score = 0
for geowk in geowks:
    answer = get_answer(conn, geoworksite=geowk, siret_truth=geowk.siret, geo_threshold=geo_threshold)
    print("----")
    print(geowk.name)
    if answer:
        score += 1
        print("{} - {} - {}".format(answer["_score"], geowk.siret, answer["_source"]["nom_complet"]))
    else:
        print("no match")

print(f"\n\nSCORE OF RETRIEVAL: {score/size_test*100}%")

In [ ]:
pd.options.display.max_rows= 100

In [ ]:
df[10:11]

In [ ]:
#request_elastic(conn, geowks[0])